#Travelling Thief Problem

Resolvendo o TSP e KP por heurística.

Neste algoritmo o problema foi resolvido seguindo o fluxo:


1.   Determinar a rota ótima (TSP) com a heurística CLKH
2.   Determinar o Picking Plan (KP) fazendo um Score em função do lucro/peso/distância
3.   Cálculo do Lucro seguindo fazendo a coleta KP no TSP





# Preparação
Procedimento de carregamento das bibliotecas e ETL dos dados

In [1]:
#Bibliotecas
import pandas as pd
import re
import math
from mip import *

## Leitura do arquivo
Carrregando o arquivo e organizando em DataFrames

In [2]:

# Função para ler o arquivo a partir da linha 280
def ler_arquivo_a_partir_da_linha(arquivo, linha_inicial):
    with open(arquivo, 'r') as file:
      cabecalho = file.readlines()[0:8]
    with open(arquivo, 'r') as file:
      cidades = file.readlines()[9:linha_inicial+10]
    with open(arquivo, 'r') as file:
      itens = file.readlines()[linha_inicial+10:]
    return cabecalho , cidades , itens

def ler_linha_específica(arquivo, numero_linha):
    with open(arquivo, 'r') as file:
        linhas = file.readlines()
        if 1 <= numero_linha <= len(linhas):
            return linhas[numero_linha - 1]


# Função para criar uma tabela (DataFrame) a partir das linhas lidas
def criar_tabela(linhas):
    # Supondo que os dados estejam separados por tabulações
    dados = [linhas[i].strip().split('\t') for i in range(len(linhas))]
    # Criar DataFrame
    df = pd.DataFrame(dados)
    df.columns = df.iloc[0] # determinando cabeçalho
    df = df[1:] # Excuindo primeira linha
    df = df.applymap(pd.to_numeric, errors='coerce') # Convertendo para inteiro

    return df

# Caminho do arquivo de texto
arquivo = 'eil51_n50_bounded-strongly-corr_01.txt'

# Paramêtros
total_cidades = int("".join(filter(str.isdigit,ler_linha_específica(arquivo,3))))
n_item = int("".join(filter(str.isdigit,ler_linha_específica(arquivo,4))))
capacity = int("".join(filter(str.isdigit,ler_linha_específica(arquivo,5))))
min_speed = float(re.findall(r'\d+\.\d+',ler_linha_específica(arquivo,6))[0])
max_speed = 1
rent_ratio = float(re.findall(r'\d+\.\d+',ler_linha_específica(arquivo,8))[0])

# Ler o arquivo a partir da linha 280
cabecalho , cidades , itens = ler_arquivo_a_partir_da_linha(arquivo, total_cidades)

# Criar a tabela
cidades = [i.replace('NODE_COORD_SECTION	(INDEX, X, Y): ', 'cidade\tx\ty') for i in cidades]
cidades = criar_tabela(cidades)


itens = [i.replace('ITEMS SECTION	(INDEX, PROFIT, WEIGHT, ASSIGNED NODE NUMBER): ', 'item\tprofit\tweight\tcidade') for i in itens]
itens = criar_tabela(itens)

# Exibir a tabela
cidades.head()


C:\Users\lucas\AppData\Local\Temp\ipykernel_9252\4133895025.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(pd.to_numeric, errors='coerce') # Convertendo para inteiro


,cidade,x,y
1,1,37,52
2,2,49,49
3,3,52,64
4,4,20,26
5,5,40,30


## Calculando a distância entre as cidades
Utilizando a distância euclidiana foi feito o calculo das distâncias e organizando em uma matriz.

In [3]:
def calcular_distancia(cidade1, cidade2):
    return math.sqrt((cidade2[1] - cidade1[1])**2 + (cidade2[2] - cidade1[2])**2)

# Criando a matriz de distâncias
n = len(cidades)
matriz_distancias = [[0] * n for _ in range(n)]

for i in range(n):
    for j in range(n):
        if i != j:
            matriz_distancias[i][j] = calcular_distancia(cidades.iloc[i], cidades.iloc[j])


C:\Users\lucas\AppData\Local\Temp\ipykernel_9252\467659222.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return math.sqrt((cidade2[1] - cidade1[1])**2 + (cidade2[2] - cidade1[2])**2)


# Resolução

## Problema do Caixeiro (isolado) - TSP
Resolvendo o problema do TSP através do método exato resolvendo com o MIP-Python

In [4]:
from itertools import product
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY

# Lista de cidades a serem visitadas
places = cidades['cidade'].tolist()

# nós (n) e vértices (V)
n, V = len(cidades), set(range(len(cidades)))

# Matriz de distâncias
c = matriz_distancias

model = Model()

# variável binária indicando da cidade
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

#variável de apoio para eliminação de subrotas
y = [model.add_var() for i in V]

# Função objetivo para minimizar a distância
model.objective = minimize(xsum(matriz_distancias[i][j]*x[i][j] for i in V for j in V))

#Restrições
## Garantir a saída da cidade uma vez
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

## Garantir a entrada na cidade uma unica vez
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

## Eliminação de subrotas
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n+1)*x[i][j] >= y[j]-n

# Resolvendo o problema com solver
model.optimize()

# Verificando solução encontrada
if model.num_solutions:
    out.write('Distância Total %g \nRota: %s'
              % (model.objective_value, places[0]))
    rota = []
    nc = 0
    while True:
        nc = [i for i in V if x[nc][i].x >= 0.99][0]
        out.write(' -> %s' % places[nc])
        rota.append(places[nc])

        if nc == 0:
            break
    out.write('\n')

Distância Total 428.872 
Rota: 1 -> 22 -> 8 -> 26 -> 31 -> 28 -> 3 -> 36 -> 35 -> 20 -> 29 -> 2 -> 16 -> 50 -> 21 -> 34 -> 30 -> 9 -> 49 -> 10 -> 39 -> 33 -> 45 -> 15 -> 44 -> 42 -> 19 -> 40 -> 41 -> 13 -> 25 -> 14 -> 24 -> 43 -> 7 -> 23 -> 48 -> 6 -> 27 -> 51 -> 46 -> 12 -> 47 -> 18 -> 4 -> 17 -> 37 -> 5 -> 38 -> 11 -> 32 -> 1


## Problema da Mochila (isolado) - KP
Resolvendo o problema da mochila pelo método exato utilizando o MIP-Python

In [ ]:
profit = itens['profit'].tolist()
weight = itens['weight'].tolist()

#Instânciando o modelo
m = Model("knapsack")

x = [m.add_var(var_type=BINARY) for i in range(n_item)]
#Função Objetivo
m.objective = maximize(xsum(profit[i] * x[i] for i in range(n_item)))

#Restrição
m += xsum(weight[i] * x[i] for i in range(n_item)) <= capacity

#Resolvendo o modelo
m.optimize()

#Resultados
x_itens = [] #vetor de itens coletados
for i in range(n_item):
  x_itens.append(x[i].x)

selected = [i for i in range(n_item) if x[i].x >= 0.99] #itens selecionados
cargo_weight = sum(weight[i] for i in selected) #peso carregado


#Imprimindo resultado
print('Ótimo {}'.format(m.objective_value, m.vars))
print("Peso Carregado: {}".format(cargo_weight))
print("Itens selecionados: {}".format(selected))

Ótimo 7124.0
Peso Carregado: 4024
Itens selecionados: [0, 1, 2, 3, 17, 18, 19, 20, 33, 42, 43, 46, 47, 48, 49]


In [5]:
#Função para elencar importancia dos itens considerando a razão
## profit/(weight*distance)

def most_important(origem, itens_mod):

  itens_mod['ratio pw']= itens_mod['profit']/itens_mod['weight']
  itens_mod['distance']= [matriz_distancias[origem][i-1] for i in itens_mod['cidade']]
  itens_mod.drop( itens_mod.loc[itens_mod['distance'] == 0].index, inplace=True)
  itens_mod['ratio pwd']= itens_mod['profit']/itens_mod['weight']/itens_mod['distance']
  itens_mod = itens_mod.sort_values(by='ratio pwd', ascending=False)
  itens_mod.reset_index(drop=True, inplace=True)
  itens_mod.head()
  return itens_mod.iloc[0]


In [31]:
kp = itens.copy()
kp['ratio pw']= kp['profit']/kp['weight']
total_weight = 0

kp_f = pd.DataFrame(columns=['item', 'cidade', 'profit', 'weight', 'ratio pw'])
rota_reversed = rota[::-1]

for cidade in rota_reversed:
  kp1 = kp[kp['cidade'] == cidade]
  kp1 = kp1.sort_values(by='ratio pw', ascending=False)
  kp1.reset_index(drop=True, inplace=True)
kp1.head()


,item,profit,weight,cidade,ratio pw


In [32]:
c

1

In [20]:
itens_mod = itens.copy()
kp['ratio pw']= itens_mod['profit']/itens_mod['weight']
total_weight = 0

#inverter a rota
rota.reverse()

while total_weight <= capacity:
    for cidade in rota:

      coleta = most_important(cidade, itens_mod) # Pass itens_mod to the function call
      rota = pd.concat([rota, pd.DataFrame(coleta).T], ignore_index=True)
      itens_mod.drop(itens_mod.loc[itens_mod['item'] == coleta['item']].index, inplace=True)
      #itens_mod.drop(itens_mod.loc[itens_mod['cidade'] == origem+1].index, inplace=True)
      #itens_mod = itens_mod[itens_mod['distance']>0]
      #itens_mod.reset_index(drop=True, inplace=True)
      #origem = int(coleta['cidade'])-1

      total_profit = rota['profit'].sum()
      total_weight = rota['weight'].sum()

if total_weight >= capacity:
  rota = rota_f.iloc[:-1]
  total_profit = rota['profit'].sum()
  total_weight = rota['weight'].sum()

  print("Total Weight - ", total_weight, "\n Total Profit - ", total_profit)

for

SyntaxError: invalid syntax (2299896993.py, line 29)

In [30]:
rota[::-1]

[22,
 8,
 26,
 31,
 28,
 3,
 36,
 35,
 20,
 29,
 2,
 16,
 50,
 21,
 34,
 30,
 9,
 49,
 10,
 39,
 33,
 45,
 15,
 44,
 42,
 19,
 40,
 41,
 13,
 25,
 14,
 24,
 43,
 7,
 23,
 48,
 6,
 27,
 51,
 46,
 12,
 47,
 18,
 4,
 17,
 37,
 5,
 38,
 11,
 32,
 1]

In [18]:
for cidade in rota:
    coleta = most_important(cidade, itens_mod)


IndexError: list index out of range

In [ ]:



capacity = 4029
origem = 0

while total_weight <= capacity:
      coleta = most_important(origem, itens_mod) # Pass itens_mod to the function call
      rota= pd.concat([rota, pd.DataFrame(coleta).T], ignore_index=True)
      itens_mod.drop(itens_mod.loc[itens_mod['item'] == coleta['item']].index, inplace=True)
      #itens_mod.drop(itens_mod.loc[itens_mod['cidade'] == origem+1].index, inplace=True)
      itens_mod = itens_mod[itens_mod['distance']>0]
      itens_mod.reset_index(drop=True, inplace=True)
      origem = int(coleta['cidade'])-1

      total_profit = rota['profit'].sum()
      total_weight = rota['weight'].sum()

if total_weight >= capacity:
  rota = rota_f.iloc[:-1]
  total_profit = rota['profit'].sum()
  total_weight = rota['weight'].sum()

  print("Total Weight - ", total_weight, "\n Total Profit - ", total_profit)

rota

## Calculando o Resultado do TTP
Dados as soluções do TSP e KP isoladamente, será calculado o resultado do TTP

In [ ]:
#Criando dataframe com os resultados de KP e TSP

TTP = pd.DataFrame(x_itens, columns=['x_itens'])
TTP.index += 1
TTP = pd.concat([itens, kp], axis=1)
TTP.set_index('cidade',inplace=True)
TTP = kp.loc[rota[:-1]]
TTP['index'] = kp.index
TTP.reset_index(drop=True, inplace=True)
TTP.rename(columns={'index':'cidade'}, inplace=True)

#Distância entre a cidade atual e da cidade anterior
TTP['distancia'] = 0
for i in range(len(TTP)):
  if i==0:
    TTP['distancia'][i] = matriz_distancias[0][int(TTP['cidade'][i])-1]
  else:
    TTP['distancia'][i] = matriz_distancias[int(TTP['cidade'][i-1]-1)][int(TTP['cidade'][i])-1]

TTP.head()

C:\Users\lucas\AppData\Local\Temp\ipykernel_13996\786744291.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  kp['distancia'][i] = matriz_distancias[0][int(kp['cidade'][i])-1]
C:\Users\lucas\AppData\Local\Temp\ipykernel_13996\786744291.py

,item,profit,weight,x_itens,cidade,distancia
0,21,666,366,1.0,22,7.071068
1,7,3984,3584,0.0,8,12.083046
2,25,923,823,0.0,26,7.211103
3,30,689,589,0.0,31,10.049876
4,27,563,463,0.0,28,6.324555


In [ ]:
#Função Tempo de Viagem
def tempo(xi,xi_1,vc):
  return (matriz_distancia[xi][xi_1]/Vc)

#Função Velocidade
cte_v = (max_speed - min_speed)/capacity

def velocidade(w_c):
  return max_speed - cte_v*w_c

#Função Objetivo
def FOT (xi):
  profit - rent_ratio*tempo(xi)

In [ ]:
#Calculando resultado da Função objetivo com os resultados de KP e TSP resolvidos com o método exato

w_c = 0 #peso carregado atual
g_xz = 0 #lucro - função objetivo
g_i = 0 #Lucro inicial
Px=0 #Rendimento
Pxi=0 #Rendimento inicial
RT=0 #Aluguel da mochila
RTi=0 #Aluguel da mochila inicial

for i in range(total_cidades-1):
  if i == 0:
    g_i = 0
    Pxi = 0
    RTi = 0
  else:
    w_c = w_c + kp.weight[i]*kp.x_itens[i]
    Pxi = kp.profit[i]*kp.x_itens[i]
    RTi = rent_ratio*kp.distancia[i]/velocidade(w_c)
    g_i = Pxi - RTi
  Px = Px + Pxi
  RT = RT + RTi

  g_xz = g_xz + g_i

print("Receita - ", Px)
print("Despesa - ", RT)
print("Lucro - ", g_xz)

Receita -  6458.0
Despesa -  4798.810879373334
Lucro -  1659.1891206266644


In [ ]:
rota.sort_values(by='item', ascending=True)